In [1]:
import numpy as np
import json
from torch2trt import TRTModule
import torch
import numpy as np
import time

In [2]:
def transform(keypoint2D):
    import pdb
    kpt2D = np.vstack((keypoint2D[5:20,:2], keypoint2D[26:,:2]))
    xmin, ymin = np.min(kpt2D, axis=0)
    xmax, ymax = np.max(kpt2D, axis=0)
    w = xmax - xmin
    h = ymax - ymin
    xc = 0.5 * (xmin + xmax)
    yc = 0.5 * (ymin + ymax)
    D = np.max([w,h])

    bnd = [0.225, 0.425]

    if D < bnd[0]:
        s = bnd[0] / D
    elif D > bnd[1]:
        s = bnd[1] / D
    else:
        s = 1
    
    if D < bnd[0] or D > bnd[1]:
        kpt2D[:,0] -= xc
        kpt2D[:,1] -= yc
        kpt2D[:,:2] *= s
        kpt2D[:,0] += xc
        kpt2D[:,1] += yc
      
    return kpt2D

In [3]:
def time_synchronized():
    torch.cuda.synchronize() if torch.cuda.is_available() else None
    return time.time()

In [4]:
camera_params =[1969.23076923, 1961.85286104,  960.,  540.]

In [5]:
with open('./fencing_keypoints_2d.json') as file:
    keypoints2D = np.array(json.load(file))

In [6]:
kpts2D = np.zeros(keypoints2D.shape)
kpts2D = kpts2D[:,:,:2]

In [7]:
kpts2D[:,:,0] = (keypoints2D[:,:,0]-camera_params[2])/camera_params[0]
kpts2D[:,:,1] = (keypoints2D[:,:,1]-camera_params[3])/camera_params[1]

In [8]:
Pose3D = TRTModule()
Pose3D.load_state_dict(torch.load('../VideoPose3D/pretrained/PoseRescaleReducedTRT-2048.pth'))

<All keys matched successfully>

In [15]:
keypoints3D = []

In [16]:
t1 = time_synchronized()
for kpt2D in kpts2D:
    kpt2D = torch.tensor(np.array([[transform(kpt2D)]]), dtype=torch.float32).cuda()
    kpt3D = Pose3D(kpt2D)[0,0].cpu().numpy().astype(float)
    keypoints3D.append(kpt3D.tolist())

t2 = time_synchronized()

In [20]:
with open('./fencing_keypoints_3d.json', 'w') as file:
    json.dump(keypoints3D, file)

In [21]:
keypoints2D = np.array(keypoints2D)
keypoints3D = np.array(keypoints3D)

In [22]:
np.save('fencing_keypoints_2d.npy', keypoints2D)
np.save('fencing_keypoints_3d.npy', keypoints3D)

In [25]:
keypoints3D[-1]

array([[ 9.33609728e-04, -5.87739669e-06, -3.72585737e-06],
       [ 1.09380305e-01,  5.97786531e-03,  1.19130192e-02],
       [ 8.14566091e-02,  3.34615469e-01, -7.39836246e-02],
       [ 4.83515412e-02,  6.75919414e-01,  8.60959217e-02],
       [-1.09367795e-01, -5.86193334e-03, -1.18964622e-02],
       [-7.07633421e-02,  3.32988441e-01, -1.07303336e-01],
       [-9.71707404e-02,  6.63341403e-01,  8.17327052e-02],
       [ 1.66111700e-02, -2.39653587e-01, -4.12053168e-02],
       [ 1.31733995e-02, -4.85320240e-01, -1.20475404e-01],
       [ 2.63327416e-02, -5.53035200e-01, -1.77502900e-01],
       [ 3.79503444e-02, -6.84004486e-01, -1.85202152e-01],
       [ 1.57623485e-01, -4.35005307e-01, -8.39643851e-02],
       [ 2.05729783e-01, -2.09433958e-01,  8.11357349e-02],
       [ 2.03399435e-01,  2.37820130e-02,  1.61895752e-01],
       [-1.35796621e-01, -4.08863395e-01, -1.33981526e-01],
       [-1.93884254e-01, -1.56763196e-01, -1.21074356e-01],
       [-1.77917689e-01,  5.99206500e-02